In [1]:
import numpy as np
import pandas as pd

In [2]:
pitcher= pd.read_csv('개인투수.csv',encoding= "utf-8")
pitcher.head()

,Unnamed: 0,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,RELIEF_CK,...,P2_WHIP_RT,CB_WHIP_RT,GYEAR,NAME,POSITION,AGE_VA,MONEY,GWEEK,STADIUM,HOME
0,0,20160401HHLG0,20160401,LG,HH,0,B,60146,0,1,...,1.000000,1.50,2016,이승현,투,25.0,3700만원,금,잠실,1
1,1,20160401HHLG0,20160401,LG,HH,0,B,61891,0,1,...,1.800000,1.50,2016,임정우,투,25.0,12500만원,금,잠실,1
2,2,20160401HHLG0,20160401,LG,HH,0,B,62146,0,1,...,3.000000,0.00,2016,최성훈,투,27.0,3500만원,금,잠실,1
3,3,20160401HHLG0,20160401,LG,HH,0,B,62698,1,0,...,1.588235,3.75,2016,소사,투,31.0,500000달러,금,잠실,1
4,4,20160401HHLG0,20160401,HH,LG,0,T,65764,0,1,...,1.500000,0.00,2016,김민우,투,21.0,3800만원,금,잠실,0


In [3]:
pitcher[pitcher['INN2']==0]['INN2'] #INN2가 0인 행 확인

38       0
47       0
57       0
90       0
100      0
        ..
27533    0
27559    0
27630    0
27671    0
27697    0
Name: INN2, Length: 989, dtype: int64

In [4]:
#ERA처리; 나중에 이부분 지우고 합치기
pitcher['ERA']= 9*pitcher['ER']/(pitcher['INN2']/3)
pitcher.loc[pitcher['INN2']==0, 'ERA']=np.nan #INN2가 0인 행은 모두 nan처리
pitcher.loc[pitcher['INN2']==0, 'ERA']

38      NaN
47      NaN
57      NaN
90      NaN
100     NaN
         ..
27533   NaN
27559   NaN
27630   NaN
27671   NaN
27697   NaN
Name: ERA, Length: 989, dtype: float64

In [5]:
#7. DIPS의 개량된 형태 중 하나인 'DICE' 열 추가
pitcher['DICE']= 3.00 + (13* pitcher['HR'] + 3*( pitcher['BB']+ pitcher['HP'])-2* pitcher['KK'])/( pitcher['INN2'] /3 )
pitcher.loc[pitcher['INN2']==0, 'DICE']=np.nan

In [6]:
#8-1. FIP 계산

In [7]:
#8-1 FIP 추가 (C=3.20)
##여기서 C를 어떻게 계산해야할지...

pitcher['FIP'] = 3.20 + (13* pitcher['HR'] + 3*(pitcher['BB']+pitcher['HP']) - 2* pitcher['KK'])/ (pitcher['INN2']/3)
pitcher.loc[pitcher['INN2']==0, 'FIP'] = np.nan

In [8]:
#8-2. '한국프로야구에서 FIP 계수의 추정' 에서 제시한 kFIP 추가: (C=3.20)
pitcher['kFIP'] = 3.20 + (14* pitcher['HR'] + 3*(pitcher['BB']+pitcher['HP']) - pitcher['KK'])/ (pitcher['INN2']/3)
pitcher.loc[pitcher['INN2']==0, 'kFIP'] = np.nan

In [9]:
#9. RA9 (Run Allowed Per 9 Innings Pitched; 9이닝당 평균실점) 추가
pitcher['RA9'] = 9 * pitcher['R'] / (pitcher['INN2'] / 3)
pitcher.loc[pitcher['INN2']==0, 'RA9']=np.nan
pitcher.loc[pitcher['INN2']==0, 'RA9']

38      NaN
47      NaN
57      NaN
90      NaN
100     NaN
         ..
27533   NaN
27559   NaN
27630   NaN
27671   NaN
27697   NaN
Name: RA9, Length: 989, dtype: float64

In [10]:
#10. HR/9 (Home Runs Allowed Per 9 Innings Pitched;9이닝당 피홈런 개수) 추가
pitcher['HR/9'] = 9 * pitcher['HR'] / (pitcher['INN2'] / 3)
pitcher.loc[pitcher['INN2']==0, 'HR/9']=np.nan
pitcher.loc[pitcher['INN2']==0, 'HR/9']

38      NaN
47      NaN
57      NaN
90      NaN
100     NaN
         ..
27533   NaN
27559   NaN
27630   NaN
27671   NaN
27697   NaN
Name: HR/9, Length: 989, dtype: float64

In [11]:
#11. H/9 (Hits Runs Allowed Per 9 Innings Pitched; 9이닝당 피안타 개수)
pitcher['H/9'] = 9 * pitcher['HIT'] / (pitcher['INN2'] / 3)
pitcher.loc[pitcher['INN2']==0, 'H/9']=np.nan
pitcher.loc[pitcher['INN2']==0, 'H/9']

38      NaN
47      NaN
57      NaN
90      NaN
100     NaN
         ..
27533   NaN
27559   NaN
27630   NaN
27671   NaN
27697   NaN
Name: H/9, Length: 989, dtype: float64

In [12]:
#OAVG/OOBP/OSLG/OOPS – 피안타율/피출루율/피장타율/피OPS 계산 전 분모가 0이 되는 행들이 존재하는지 체크.

print(len(pitcher.loc[pitcher['AB']==0]))
print(len(pitcher.loc[pitcher['AB'] + pitcher['BB'] + pitcher['HP'] + pitcher['SF']==0]))

293
25


In [13]:
#12. OAVG/OOBP/OSLG/OOPS – 피안타율/피출루율/피장타율/피OPS
pitcher['OAVG'] = pitcher['HIT']/ pitcher['AB']
pitcher.loc[pitcher['AB']==0, 'OAVG'] = np.nan

pitcher['OOBP'] = (pitcher['HIT'] + pitcher['BB'] + pitcher['HP'] ) / (pitcher['AB'] + pitcher['BB'] + pitcher['HP'] + pitcher['SF'])
pitcher.loc[pitcher['AB'] + pitcher['BB'] + pitcher['HP'] + pitcher['SF']==0, 'OOBP'] ==np.nan

pitcher['OSLG'] = ( pitcher['HIT'] + 2*pitcher['H2'] + 3*pitcher['H3']+ 4*pitcher['HR'])/pitcher['AB']
pitcher.loc[pitcher['AB']==0, 'OSLG'] = np.nan

pitcher['OOPS'] = pitcher['OOBP']+ pitcher['OSLG']


In [14]:
pitcher

,Unnamed: 0,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,RELIEF_CK,...,DICE,FIP,kFIP,RA9,HR/9,H/9,OAVG,OOBP,OSLG,OOPS
0,0,20160401HHLG0,20160401,LG,HH,0,B,60146,0,1,...,1.500000,1.700000,3.200000,0.00,0.0,4.50,0.142857,0.250000,0.142857,0.392857
1,1,20160401HHLG0,20160401,LG,HH,0,B,61891,0,1,...,3.600000,3.800000,4.400000,0.00,0.0,10.80,0.333333,0.428571,0.333333,0.761905
2,2,20160401HHLG0,20160401,LG,HH,0,B,62146,0,1,...,3.000000,3.200000,3.200000,0.00,0.0,27.00,0.500000,0.500000,0.500000,1.000000
3,3,20160401HHLG0,20160401,LG,HH,0,B,62698,1,0,...,2.166667,2.366667,3.033333,6.00,0.0,13.50,0.360000,0.384615,0.520000,0.904615
4,4,20160401HHLG0,20160401,HH,LG,0,T,65764,0,1,...,3.750000,3.950000,4.700000,6.75,0.0,6.75,0.250000,0.400000,0.750000,1.150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27799,27799,20200719WOSK0,20200719,WO,SK,0,T,67313,1,0,...,0.800000,1.000000,2.400000,0.00,0.0,7.20,0.222222,0.263158,0.222222,0.485380
27800,27800,20200719WOSK0,20200719,WO,SK,0,T,67391,0,1,...,9.000000,9.200000,10.700000,0.00,0.0,0.00,0.000000,0.500000,0.000000,0.500000
27801,27801,20200719WOSK0,20200719,WO,SK,0,T,68341,0,1,...,10.500000,10.700000,13.700000,54.00,0.0,13.50,0.250000,0.571429,0.250000,0.821429
27802,27802,20200719WOSK0,20200719,WO,SK,0,T,69399,0,1,...,3.000000,3.200000,3.200000,0.00,0.0,27.00,0.500000,0.500000,0.500000,1.000000
